In [1]:
# The following notebook is to be run in kaggle. Upload the data from 'FaceDetection1' to your kaggle datasets. 
# Kaggle is used since it provides GPU. So change your accelerator to GPU preferably GPU T4 x2

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1.Setup and Get Data

## 1.1 Install Dependencies and Setup

In [ ]:
!pip install labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations

## 1.2 Collect Images Using OpenCV

In [ ]:
import os
import time
import uuid
import cv2

## 1.3 Annotate Images with LabelMe

# 2. Review Dataset and Build Image Loading Function

## 2.1 Import TF and Deps

In [ ]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt

## 2.2 Limit GPU Memory Growth

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

## 2.3 Load Image into TF Data Pipeline

In [ ]:
images = tf.data.Dataset.list_files('/kaggle/input/mydata/data2/images/*.jpg')

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
images = images.map(load_image)

In [ ]:
images.as_numpy_iterator().next()

In [ ]:
type(images)

## 2.4 View Raw Images with Matplotlib

In [ ]:
image_generator = images.batch(4).as_numpy_iterator()

In [ ]:
plot_images = image_generator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, image in enumerate(plot_images):
    ax[idx].imshow(image) 
plt.show()

# 3. Partition Unaugmented Data

## 3.1 MANUALLY SPLT DATA INTO TRAIN TEST AND VAL

In [ ]:
180*0.7 # 126 for train

In [ ]:
180*0.15 # 27 for test and 27 for val

## 3.2 Move the Matching Labels

# 4. Apply Image Augmentation on Images and Labels using Albumentations

## 4.1 Setup Albumentations Transform Pipeline

In [ ]:
import albumentations as alb

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450), 
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='albumentations', 
                                                  label_fields=['class_labels']))

# 5. Build and Run Augmentation Pipeline

## 5.1 Run Augmentation Pipeline

In [ ]:
os.makedirs('/kaggle/working/aug_data/train/Tarun')
os.makedirs('/kaggle/working/aug_data/train/Nihaal')

In [ ]:
os.makedirs('/kaggle/working/aug_data/test/Tarun')
os.makedirs('/kaggle/working/aug_data/test/Nihaal')

In [ ]:
os.makedirs('/kaggle/working/aug_data/val/Tarun')
os.makedirs('/kaggle/working/aug_data/val/Nihaal')

In [ ]:
for partition in ['Tarun','Nihaal']: 
    for image in os.listdir(os.path.join('/kaggle/input/mydata/D/train',partition)):
        img = cv2.imread(os.path.join('/kaggle/input/mydata/D/train',partition,image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('/kaggle/input/mydata/D/train',partition+'Labels',f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [640,480,640,480]))

        try: 
            for x in range(60):
                augmented = augmentor(image=img, bboxes=[coords], class_labels=[label['shapes'][0]['label']])
                cv2.imwrite(os.path.join('/kaggle/working/aug_data/train', partition, f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])
        except Exception as e:
            print(e)

In [ ]:
for partition in ['Tarun','Nihaal']: 
    for image in os.listdir(os.path.join('/kaggle/input/mydata/D/test',partition)):
        img = cv2.imread(os.path.join('/kaggle/input/mydata/D/test',partition,image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('/kaggle/input/mydata/D/test',partition+'Labels',f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [640,480,640,480]))

        try: 
            for x in range(60):
                augmented = augmentor(image=img, bboxes=[coords], class_labels=[label['shapes'][0]['label']])
                cv2.imwrite(os.path.join('/kaggle/working/aug_data/test', partition, f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])
        except Exception as e:
            print(e)

In [ ]:
for partition in ['Tarun','Nihaal']: 
    for image in os.listdir(os.path.join('/kaggle/input/mydata/D/val',partition)):
        img = cv2.imread(os.path.join('/kaggle/input/mydata/D/val',partition,image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('/kaggle/input/mydata/D/val',partition+'Labels',f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [640,480,640,480]))

        try: 
            for x in range(60):
                augmented = augmentor(image=img, bboxes=[coords], class_labels=[label['shapes'][0]['label']])
                cv2.imwrite(os.path.join('/kaggle/working/aug_data/val', partition, f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])
        except Exception as e:
            print(e)

# 6. Model Building

In [ ]:
from keras.applications import vgg16

In [ ]:
# pre trained weights : create my model
model = vgg16.VGG16(weights='imagenet')

In [ ]:
for layer in model.layers:
    layer.trainable=False

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
top_model=model.output
top_model=Dense(1024, activation='relu')(top_model)
top_model= Dense(512, activation='relu')(top_model)
top_model= Dense(2, activation='softmax')(top_model)

In [ ]:
from keras.models import Model

In [ ]:
newmodel=Model(inputs=model.input,outputs=top_model)

In [ ]:
newmodel.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/kaggle/working/aug_data/train/'
validation_data_dir = '/kaggle/working/aug_data/val/'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 100
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

In [ ]:
newmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
newmodel.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator,
        )

In [ ]:
newmodel.save('faceClassify.h5')

In [ ]:
from IPython.display import FileLink
FileLink(r'faceClassify.h5')